In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [1]:
import openai
import pandas as pd

import mlflow
mlflow.set_tracking_uri(uri="http://localhost:8082")

In [2]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

In [4]:
eval_df

,inputs,ground_truth
0,How does useEffect() work?,The useEffect() hook tells React that your com...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...


In [3]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics


2025/02/07 14:02:40 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/02/07 14:02:40 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/02/07 14:02:43 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/07 14:02:52 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:02:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:02:52 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:02:52 WARNING mlf

🏃 View run clean-bass-332 at: http://localhost:8082/#/experiments/0/runs/12ab764b896b48998f6dc55cdd49155d
🧪 View experiment at: http://localhost:8082/#/experiments/0


{'exact_match/v1': 0.0}

[Trace(request_id=5bb2f53425074a5e834f9f6856194676), Trace(request_id=a66dd75847214b0882990f6d2da37491), Trace(request_id=0fa0739ac35841afb2ce46dfccdd8b9b), Trace(request_id=cf00407e1e6c4dfebf5c08fe0feb9da3)]

In [5]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,67
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function indicates tha...,62
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used in excep...,64
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the use of multiple p...,89


In [12]:
len(results.tables["eval_results_table"].iloc[0].outputs.split(" "))

56

In [13]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

In [16]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/02/07 14:31:26 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/02/07 14:31:33 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/07 14:31:33 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:31:33 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:31:33 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:31:33 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:31:33 WARNING mlflow.metrics.metric_definitions: Failed to

  0%|          | 0/1 [00:00<?, ?it/s]

2025/02/07 14:31:38 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:31:38 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:31:38 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:31:38 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:31:38 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:31:38 WARNING mlflow.models.evaluation.utils.metri

  0%|          | 0/4 [00:00<?, ?it/s]

🏃 View run nosy-skunk-609 at: http://localhost:8082/#/experiments/0/runs/9e3196df14d24cb9bf82b2a9a5d0f33e
🧪 View experiment at: http://localhost:8082/#/experiments/0


{'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(4.0),
 'answer_similarity/v1/variance': np.float64(1.5),
 'answer_similarity/v1/p90': np.float64(5.0)}

[Trace(request_id=d48468dae15b4803a4b6f0cb5367f6ab), Trace(request_id=78f9b9035f454e08bf9c72892e4c44dc), Trace(request_id=fd31caaa76474ba485b2c5ed0083af15), Trace(request_id=2147c1d149964bcc8a8a5373d664426d)]

In [17]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,83,5,The output closely aligns with the provided ta...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function indicates t...,68,2,The output provides a correct explanation of t...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to speci...,52,5,The model's output closely aligns with the pro...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves using multiple proces...,71,4,The output aligns with the provided targets in...


In [18]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

In [19]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2025/02/07 14:46:43 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/02/07 14:46:45 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/07 14:46:45 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:46:45 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:46:45 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:46:45 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:46:45 WARNING mlflow.metrics.metric_definitions: Failed to

  0%|          | 0/1 [00:00<?, ?it/s]

2025/02/07 14:46:48 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:46:48 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:46:48 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:46:48 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:46:48 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:46:48 WARNING mlflow.models.evaluation.utils.metri

  0%|          | 0/4 [00:00<?, ?it/s]

🏃 View run delightful-shark-523 at: http://localhost:8082/#/experiments/0/runs/a779622210664cb08ed59cde4feb1c92
🧪 View experiment at: http://localhost:8082/#/experiments/0
{'professionalism/v1/mean': np.float64(4.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(4.0)}


[Trace(request_id=5d6a840a7beb48e39de5edccf316b712), Trace(request_id=aec569d67fb54a64b9c024f4d513ea99), Trace(request_id=a7932672a5fb4295b32afa3c0e57dd57), Trace(request_id=cf3a22de56284f2ab316c238db222270)]

In [20]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

2025/02/07 14:48:32 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/02/07 14:48:40 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/02/07 14:48:40 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:48:40 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:48:40 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:48:40 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:48:40 WARNING mlflow.metrics.metric_definitions: Failed to

  0%|          | 0/1 [00:00<?, ?it/s]

2025/02/07 14:48:43 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/02/07 14:48:43 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:48:43 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:48:43 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/02/07 14:48:43 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/02/07 14:48:43 WARNING mlflow.models.evaluation.utils.metri

  0%|          | 0/4 [00:00<?, ?it/s]

🏃 View run gentle-wasp-817 at: http://localhost:8082/#/experiments/0/runs/b6a1ba3de7ad405aaa4f3da7201bd494
🧪 View experiment at: http://localhost:8082/#/experiments/0
{'professionalism/v1/mean': np.float64(4.75), 'professionalism/v1/variance': np.float64(0.1875), 'professionalism/v1/p90': np.float64(5.0)}


[Trace(request_id=51cd633d5aab42fcaad32c13ee447bbf), Trace(request_id=555e14ef4e964d74bca74f90a88a88ad), Trace(request_id=2226ce6e3d3143f59730cb7060fd84dc), Trace(request_id=8ca3848c2410406eaf6169c6ff79f050)]